In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import learning_curve
from sklearn.impute import SimpleImputer


In [ ]:
file_path = '/content/drive/MyDrive/Data Mining $ Machine Learning/crime_dataset_india.csv'
#file_path = 'Book2.xlsx'

In [ ]:
df = pd.read_csv(file_path)
df.head()



/usr/local/lib/python3.10/dist-packages/google/colab/_dataframe_summarizer.py:88: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  cast_date_col = pd.to_datetime(column, errors="coerce")


,Report Number,Date Reported,Date of Occurrence,Time of Occurrence,City,Crime Code,Crime Description,Victim Age,Victim Gender,Weapon Used,Crime Domain,Police Deployed,Case Closed,Date Case Closed
0,1,02-01-2020 00:00,01-01-2020 00:00,01-01-2020 01:11,Ahmedabad,576,IDENTITY THEFT,16,M,Blunt Object,Violent Crime,13,No,NaN
1,2,01-01-2020 19:00,01-01-2020 01:00,01-01-2020 06:26,Chennai,128,HOMICIDE,37,M,Poison,Other Crime,9,No,NaN
2,3,02-01-2020 05:00,01-01-2020 02:00,01-01-2020 14:30,Ludhiana,271,KIDNAPPING,48,F,Blunt Object,Other Crime,15,No,NaN
3,4,01-01-2020 05:00,01-01-2020 03:00,01-01-2020 14:46,Pune,170,BURGLARY,49,F,Firearm,Other Crime,1,Yes,29-04-2020 05:00
4,5,01-01-2020 21:00,01-01-2020 04:00,01-01-2020 16:51,Pune,421,VANDALISM,30,F,Other,Other Crime,18,Yes,08-01-2020 21:00


In [ ]:
# Check for missing values
print("Missing values in each column:")
print(df.isnull().sum())


Missing values in each column:
Report Number             0
Date Reported             0
Date of Occurrence        0
Time of Occurrence        0
City                      0
Crime Code                0
Crime Description         0
Victim Age                0
Victim Gender             0
Weapon Used            5790
Crime Domain              0
Police Deployed           0
Case Closed               0
Date Case Closed      20098
dtype: int64


In [ ]:
# Handling missing values by dropping rows with any missing values
df.dropna(inplace=True)
print(f"Dataset shape after dropping missing values: {df.shape}")


Dataset shape after dropping missing values: (17130, 14)


In [ ]:
# Encode categorical variables
label_encoders = {}
for column in df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

print("Categorical variables encoded.")


Categorical variables encoded.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.impute import SimpleImputer

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/Data Mining $ Machine Learning/crime_dataset_india.csv')

# Handling missing values using SimpleImputer
imputer = SimpleImputer(strategy='most_frequent')
df['Victim Age'] = imputer.fit_transform(df[['Victim Age']])

# Encode categorical variables
label_encoders = {}
for column in df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Select features and target
X = df[['City', 'Victim Age', 'Victim Gender', 'Weapon Used', 'Police Deployed']]
y = df['Crime Domain']

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling (important for KNN, SVM)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define individual models
knn = KNeighborsClassifier(n_neighbors=5)
svm = SVC(kernel='linear', probability=True, random_state=42)
rf = RandomForestClassifier(n_estimators=100, random_state=42)
nb = GaussianNB()

# Create an ensemble model (Voting Classifier)
ensemble = VotingClassifier(estimators=[('knn', knn), ('svm', svm), ('rf', rf), ('nb', nb)], voting='soft')

# Train and evaluate each model
models = {'KNN': knn, 'SVM': svm, 'Random Forest': rf, 'Naive Bayes': nb, 'Ensemble': ensemble}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"\n=== {name} Classification Report ===")
    print(classification_report(y_test, y_pred))
    print(f"\n=== {name} Confusion Matrix ===")
    print(confusion_matrix(y_test, y_pred))
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {accuracy:.4f}")



=== KNN Classification Report ===
              precision    recall  f1-score   support

           0       0.07      0.04      0.05       737
           1       0.57      0.78      0.66      4580
           2       0.02      0.00      0.00       393
           3       0.28      0.15      0.19      2322

    accuracy                           0.49      8032
   macro avg       0.23      0.24      0.23      8032
weighted avg       0.41      0.49      0.44      8032


=== KNN Confusion Matrix ===
[[  32  579    4  122]
 [ 254 3577   33  716]
 [  28  310    1   54]
 [ 118 1844   16  344]]
KNN Accuracy: 0.4923

=== SVM Classification Report ===
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       737
           1       0.57      1.00      0.73      4580
           2       0.00      0.00      0.00       393
           3       0.00      0.00      0.00      2322

    accuracy                           0.57      8032
   macro avg       0.14  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



=== Random Forest Classification Report ===
              precision    recall  f1-score   support

           0       0.09      0.03      0.05       737
           1       0.57      0.77      0.66      4580
           2       0.05      0.01      0.02       393
           3       0.30      0.19      0.23      2322

    accuracy                           0.50      8032
   macro avg       0.25      0.25      0.24      8032
weighted avg       0.42      0.50      0.45      8032


=== Random Forest Confusion Matrix ===
[[  24  581    4  128]
 [ 148 3532   53  847]
 [  10  319    5   59]
 [  80 1767   35  440]]
Random Forest Accuracy: 0.4981

=== Naive Bayes Classification Report ===
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       737
           1       0.57      1.00      0.73      4580
           2       0.00      0.00      0.00       393
           3       0.00      0.00      0.00      2322

    accuracy                           0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



=== Ensemble Classification Report ===
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       737
           1       0.57      0.95      0.71      4580
           2       0.00      0.00      0.00       393
           3       0.29      0.05      0.08      2322

    accuracy                           0.56      8032
   macro avg       0.21      0.25      0.20      8032
weighted avg       0.41      0.56      0.43      8032


=== Ensemble Confusion Matrix ===
[[   0  691    0   46]
 [   0 4372    0  208]
 [   0  377    0   16]
 [   2 2212    0  108]]
Ensemble Accuracy: 0.5578


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Convert "Case Closed" column to binary
df['Case Closed'] = LabelEncoder().fit_transform(df['Case Closed'])

# Select features and target
X = df[['City', 'Victim Age', 'Victim Gender', 'Weapon Used', 'Police Deployed']]
y = df['Case Closed']

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling (important for KNN, SVM)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train and evaluate each model
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"\n=== {name} Classification Report (Case Closed Prediction) ===")
    print(classification_report(y_test, y_pred))
    print(f"\n=== {name} Confusion Matrix ===")
    print(confusion_matrix(y_test, y_pred))
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy (Case Closed Prediction): {accuracy:.4f}")



=== KNN Classification Report (Case Closed Prediction) ===
              precision    recall  f1-score   support

           0       0.50      0.50      0.50      4046
           1       0.49      0.49      0.49      3986

    accuracy                           0.50      8032
   macro avg       0.49      0.49      0.49      8032
weighted avg       0.50      0.50      0.50      8032


=== KNN Confusion Matrix ===
[[2024 2022]
 [2034 1952]]
KNN Accuracy (Case Closed Prediction): 0.4950


KeyboardInterrupt: 

In [ ]:
# Assuming 'Crime Domain' represents crime severity
# Select features and target
X = df[['City', 'Victim Age', 'Victim Gender', 'Weapon Used', 'Police Deployed']]
y = df['Crime Domain']

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling (important for KNN, SVM)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train and evaluate each model
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"\n=== {name} Classification Report (Crime Severity Prediction) ===")
    print(classification_report(y_test, y_pred))
    print(f"\n=== {name} Confusion Matrix ===")
    print(confusion_matrix(y_test, y_pred))
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy (Crime Severity Prediction): {accuracy:.4f}")


In [ ]:
from sklearn.cluster import KMeans
import seaborn as sns
import matplotlib.pyplot as plt

# Group by city and count the number of crimes per city
city_crime_counts = df.groupby('City').size().reset_index(name='Crime Count')

# Apply KMeans clustering to identify hotspot areas
kmeans = KMeans(n_clusters=3, random_state=42)
city_crime_counts['Cluster'] = kmeans.fit_predict(city_crime_counts[['Crime Count']])

# Visualizing the Crime Hotspots (Clusters)
plt.figure(figsize=(10, 6))
sns.scatterplot(x='City', y='Crime Count', hue='Cluster', data=city_crime_counts, palette='viridis')
plt.title('Crime Hotspot Analysis (City-wise Clustering)')
plt.xticks(rotation=90)
plt.show()

# Display the cities in each cluster (hotspot group)
for i in range(3):
    cluster_cities = city_crime_counts[city_crime_counts['Cluster'] == i]['City'].values
    print(f"Cluster {i + 1} (Crime Hotspot Group):")
    print(cluster_cities)
    print("\n")


In [ ]:
# Convert 'Date of Occurrence' to datetime
df['Date of Occurrence'] = pd.to_datetime(df['Date of Occurrence'])

# Aggregate data by date to find the number of crimes per day
crime_trend = df.groupby('Date of Occurrence').size().reset_index(name='Crime Count')

# Sort the data by date
crime_trend = crime_trend.sort_values(by='Date of Occurrence')

# Plot the trend
plt.figure(figsize=(10, 6))
plt.plot(crime_trend['Date of Occurrence'], crime_trend['Crime Count'], color='b')
plt.title('Crime Occurrence Over Time')
plt.xlabel('Date of Occurrence')
plt.ylabel('Crime Count')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

# Prepare the data for time series prediction
crime_trend['Days Since Start'] = (crime_trend['Date of Occurrence'] - crime_trend['Date of Occurrence'].min()).dt.days
X = crime_trend[['Days Since Start']]
y = crime_trend['Crime Count']

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a simple Linear Regression model for time series prediction
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluate the model
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse:.4f}')

# Plot the predictions vs actual values
plt.figure(figsize=(10, 6))
plt.scatter(X_test, y_test, color='blue', label='Actual')
plt.plot(X_test, y_pred, color='red', label='Predicted')
plt.title('Crime Prediction Over Time')
plt.xlabel('Days Since Start')
plt.ylabel('Crime Count')
plt.legend()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/crime_dataset_india.csv')

# Handling missing values using SimpleImputer
imputer = SimpleImputer(strategy='most_frequent')
df['Victim Age'] = imputer.fit_transform(df[['Victim Age']])

# Encode categorical variables
label_encoders = {}
for column in df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Select features and target
X = df[['City', 'Victim Age', 'Victim Gender', 'Weapon Used', 'Police Deployed']]
y = df['Crime Domain']

# One-hot encode the target variable for multiclass classification
y = pd.get_dummies(y)

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))  # Multiclass classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test.values, axis=1)

# Print classification report
print("=== Deep Learning Model Classification Report ===")
print(classification_report(y_true, y_pred_classes))

# Accuracy
accuracy = accuracy_score(y_true, y_pred_classes)
print(f"Accuracy: {accuracy:.4f}")


In [ ]:
# Convert "Case Closed" column to binary
df['Case Closed'] = LabelEncoder().fit_transform(df['Case Closed'])

# Select features and target
X = df[['City', 'Victim Age', 'Victim Gender', 'Weapon Used', 'Police Deployed']]
y = df['Case Closed']

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the model for binary classification
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Print classification report
print("=== Deep Learning Model Classification Report (Case Closed Prediction) ===")
print(classification_report(y_test, y_pred))

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")


In [ ]:
# Assuming 'Crime Domain' represents crime severity
# Select features and target
X = df[['City', 'Victim Age', 'Victim Gender', 'Weapon Used', 'Police Deployed']]
y = df['Crime Domain']

# One-hot encode the target variable
y = pd.get_dummies(y)

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the model for multiclass classification
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))  # Multiclass classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test.values, axis=1)

# Print classification report
print("=== Deep Learning Model Classification Report (Crime Severity Prediction) ===")
print(classification_report(y_true, y_pred_classes))

# Accuracy
accuracy = accuracy_score(y_true, y_pred_classes)
print(f"Accuracy: {accuracy:.4f}")


In [ ]:
# Convert 'Date of Occurrence' to datetime
df['Date of Occurrence'] = pd.to_datetime(df['Date of Occurrence'])

# Aggregate data by date to find the number of crimes per day
crime_trend = df.groupby('Date of Occurrence').size().reset_index(name='Crime Count')

# Sort the data by date
crime_trend = crime_trend.sort_values(by='Date of Occurrence')

# Prepare the data for time series prediction
crime_trend['Days Since Start'] = (crime_trend['Date of Occurrence'] - crime_trend['Date of Occurrence'].min()).dt.days
X = crime_trend[['Days Since Start']]
y = crime_trend['Crime Count']

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model for regression (time series prediction)
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))  # Regression task

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model
y_pred = model.predict(X_test)

# Evaluate using RMSE
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")


In [ ]:
# Convert "Case Closed" column to binary
df['Case Closed'] = LabelEncoder().fit_transform(df['Case Closed'])

# Select features and target
X = df[['City', 'Victim Age', 'Victim Gender', 'Weapon Used', 'Police Deployed']]
y = df['Case Closed']

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the model for binary classification
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Print classification report
print("=== Deep Learning Model Classification Report (Case Closed Prediction) ===")
print(classification_report(y_test, y_pred))

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")


In [ ]:
# Assuming 'Crime Domain' represents crime severity
# Select features and target
X = df[['City', 'Victim Age', 'Victim Gender', 'Weapon Used', 'Police Deployed']]
y = df['Crime Domain']

# One-hot encode the target variable
y = pd.get_dummies(y)

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the model for multiclass classification
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))  # Multiclass classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test.values, axis=1)

# Print classification report
print("=== Deep Learning Model Classification Report (Crime Severity Prediction) ===")
print(classification_report(y_true, y_pred_classes))

# Accuracy
accuracy = accuracy_score(y_true, y_pred_classes)
print(f"Accuracy: {accuracy:.4f}")


In [ ]:
# Convert 'Date of Occurrence' to datetime
df['Date of Occurrence'] = pd.to_datetime(df['Date of Occurrence'])

# Aggregate data by date to find the number of crimes per day
crime_trend = df.groupby('Date of Occurrence').size().reset_index(name='Crime Count')

# Sort the data by date
crime_trend = crime_trend.sort_values(by='Date of Occurrence')

# Prepare the data for time series prediction
crime_trend['Days Since Start'] = (crime_trend['Date of Occurrence'] - crime_trend['Date of Occurrence'].min()).dt.days
X = crime_trend[['Days Since Start']]
y = crime_trend['Crime Count']

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model for regression (time series prediction)
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))  # Regression task

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model
y_pred = model.predict(X_test)

# Evaluate using RMSE
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
